# 紫微斗数 RAG 工作流 (Qdrant + LangChain)
本笔记本演示从命盘生成检索查询、压缩检索结果、生成摘要，再基于摘要回答问题的完整流程。

In [1]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Iterable, List, Dict, Any

from dotenv import load_dotenv
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLSectionSplitter
from langchain.storage import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.retrievers import ParentDocumentRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from astro_chart import full_chart_generation


/home/mameuio/miniconda3/LLM/zhiweidoushu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 环境与向量检索配置
load_dotenv()

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "zhiwei_DAG"
EMBEDDING_MODEL_HANDLE = "jinaai/jina-embeddings-v2-base-zh"
SPARSE_MODEL_HANDLE = "Qdrant/BM25"
STORE_PATH = "./store_location"

embeddings = FastEmbedEmbeddings(model_name=EMBEDDING_MODEL_HANDLE)
sparse_embeddings = FastEmbedSparse(model_name=SPARSE_MODEL_HANDLE)

client = QdrantClient(url=QDRANT_URL, prefer_grpc=True)
vectorstore = QdrantVectorStore(
    embedding=embeddings,
    client=client,
    collection_name=COLLECTION_NAME,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID
)

fs = LocalFileStore(STORE_PATH)
doc_store = create_kv_docstore(fs)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=doc_store,
    child_splitter=child_splitter,
    search_kwargs={"k": 10}
)

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever, base_compressor=compressor
)


INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:10<00:00, 9.76MiB/s]


## 选填：如需首次构建父子文档，可取消注释后运行

In [ ]:
# html_path = Path('zhiwei book.html')
# if html_path.exists():
#     text_html = html_path.read_text(encoding='utf-8')
#     headers_to_split_on = [ ("h1", "Header 1") ]
#     html_splitter = HTMLSectionSplitter(headers_to_split_on)
#     parent_docs = html_splitter.split_text(text_html)
#     retriever.add_documents(parent_docs)
# else:
#     raise FileNotFoundError('缺少 zhiwei book.html，请确认路径。')


In [3]:
def pretty_print_docs(docs, save_path: str | None = None) -> str:
    doc_text = "\n{}\n".format('-' * 100).join(
        [f"Document {i + 1}:\n{d.page_content}" for i, d in enumerate(docs)]
    )
    if save_path:
        with open(save_path, 'w', encoding='utf-8') as f_out:
            f_out.write(doc_text)
    return doc_text


In [4]:
# LLM 与提示模板
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

query_prompt_text = """
角色: 你是一个专业的紫微斗数检索查询生成器，用于 RAG(检索增强生成)。

目标: 基于用户的命盘关键信息与具体问题，生成若干高质量、可直接用于检索的查询字符串，覆盖问题最核心的宫位与关键星曜组合，并按相关性排序。

约束:
- 仅使用用户的命盘中出现的宫位、主星、辅星、煞星；不要编造。
- 若用户的命盘未提供四化，不要构造四化查询；若提供，至少包含一条带四化的查询。
- 使用标准中文术语与命名；短语之间用空格分隔；避免冗余词。
- 每条查询尽量精炼但信息完整；生成 1-3 条，按相关性从高到低排序。
- 仅输出 JSON 字符串数组；不要任何解释、标题或 Markdown 代码块。

步骤:
1) 解析问题与领域（如事业财运、感情婚姻、健康、人际等），确定最核心宫位。
2) 从命盘中提取该核心宫位及其三方四正的关键主星、**如果宮位沒有核心（空宮）主星，用對面宮位的主星來代替**, 重要辅星（左辅、右弼、文昌、文曲）、煞星（擎羊、陀罗、火星、铃星、地空、地劫。
3) 组合查询，覆盖核心宫位、关键星曜影响，去重并按相关性排序。

查询模板（仅使用以下几类，其一或多条）:
- 模式A: 宫位-星曜-主题
  形式: [宫位] [星曜组合] [问题领域] 解释
  例: 夫妻宫 廉贞七杀 感情婚姻 解释
- 模式B: 星曜-四化-宫位
  形式: [星曜] 化[禄/权/科/忌]入 [宫位] 对[问题领域]的影响
  例: 太阳化忌入父母宫 对学业的影响
- 模式C: 宫位-煞/辅
  形式: [宫位] 遇 [煞星/辅星] 作用
  例: 命宫 擎羊同宫 影响
- 模式D: 宫位关系/格局
  形式: [宫位A] [宫位B] [关系类型] 影响
      或 [星曜组合] [宫位] [格局名称] 格局
  例: 命宫 迁移宫 对照 影响
      紫微破军在丑未宫 紫府朝垣格

输入:
- 用户命盘: {user_chart}
- 用户问题: {user_question}

输出（严格遵守）:
- 仅输出 JSON 字符串数组；不含任何解释、前后缀、注释或代码块标记。
- 项目去重、语义多样化、按相关性降序。

示例输出:
[\"夫妻宫 廉贞七杀 感情婚姻 解释\",
 \"廉贞化忌入夫妻宫 对婚姻的影响\",
 \"夫妻宫三方四正 福德宫 天府 影响\",
 \"命宫 紫微 感情观\",
 \"七杀星 感情 特质\"]
"""

summary_prompt_text = """
角色: 你是专业的紫微斗数顾问，任务是基于检索到的文段整理关键信息，供后续回答使用。

目标: 在不直接回答用户问题的前提下，归纳文段中与命盘和问题相关的要点。

输入:
- 用户命盘: {user_chart}
- 用户问题: {user_question}
- 检索文段: {retrieved_passages}

指令:
1. 阅读全部文段，挑出与用户问题及命盘相关的关键信息；忽略无关材料。
2. 将信息按主题分组，并以精炼语句概述
3. 指出这些要点与命盘要素（宫位、主星、三方四正、四化等）的关联，或说明缺少对应信息。
4. 如存在重要空缺或需要额外检索的线索，单独列出。
5. 不做结论或建议；不要编造命盘或文段外的信息。

输出格式:
关键信息:
- … 
- …
命盘关联:
- …
- …
信息缺口:
- …
"""

answer_prompt_text = """
角色: 你是专业紫微斗数顾问，需要结合摘要要点与命盘回答用户问题。

输入:
- 用户命盘: {user_chart}
- 用户问题: {user_question}
- 摘要要点: {summary_text}

指令:
1. 基于摘要要点,结合命盘结构说明关键影响,如果摘要的信息没办法回答请用你多年紫薇斗数的经验来回答问题
2. 明确回答用户问题，分条阐述理由。
3. 请尽量给出谨慎、可执行的建议
4. 全文使用中文，条理清晰。

输出格式:
最终结论: …
关键理由:
- …
- …
建议:
- …
"""

query_prompt = PromptTemplate(
    input_variables=["user_question", "user_chart"],
    template=query_prompt_text
)
summary_prompt = PromptTemplate(
    input_variables=["user_question", "user_chart", "retrieved_passages"],
    template=summary_prompt_text
)
answer_prompt = PromptTemplate(
    input_variables=["user_question", "user_chart", "summary_text"],
    template=answer_prompt_text
)

query_chain = query_prompt | llm
summary_chain = summary_prompt | llm
answer_chain = answer_prompt | llm


In [5]:
def parse_json_list(raw_text: str) -> List[str]:
    start = raw_text.find('[')
    end = raw_text.rfind(']')
    if start == -1 or end == -1:
        return []
    try:
        return json.loads(raw_text[start:end + 1])
    except json.JSONDecodeError:
        return []

def generate_queries(question: str, user_chart: Dict[str, Any], top_n: int = 3) -> List[str]:
    ai_message = query_chain.invoke({"user_question": question, "user_chart": user_chart})
    queries = parse_json_list(ai_message.content)
    return queries[:top_n] if top_n else queries

def retrieve_documents(queries: Iterable[str]) -> List[Any]:
    seen_doc_ids = set()
    aggregated = []
    for q in queries:
        docs = compression_retriever.invoke(q)
        for doc in docs:
            doc_id = doc.metadata.get('doc_id') or doc.metadata.get('source') or doc.metadata.get('id')
            key = doc_id or id(doc)
            if key in seen_doc_ids:
                continue
            seen_doc_ids.add(key)
            aggregated.append(doc)
    summarized_docs = pretty_print_docs(aggregated)
    return summarized_docs

def summarize_passages(question: str, user_chart: Dict[str, Any], passages: str) -> str:
    payload = {
        "user_question": question,
        "user_chart": user_chart,
        "retrieved_passages": passages
    }
    ai_message = summary_chain.invoke(payload)
    return ai_message.content

def answer_question(question: str, user_chart: Dict[str, Any], summary_text: str) -> str:
    payload = {
        "user_question": question,
        "user_chart": user_chart,
        "summary_text": summary_text
    }
    ai_message = answer_chain.invoke(payload)
    return ai_message.content


In [7]:
# 示例：完整运行一遍流程
question = "我什么时候能遇到正缘？"
user_chart = full_chart_generation("1994-8-25", 11, "男")

queries = generate_queries(question, user_chart, top_n=5)
print('生成的检索查询:', queries)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


生成的检索查询: ['夫妻宫 天钺 感情婚姻 解释', '命宫 廉贞破军 感情婚姻 解释', '夫妻宫 遇 天钺 作用']


In [8]:
retrieved_docs = retrieve_documents(queries)

In [10]:
print(retrieved_docs)

Document 1:
☆11、天相星(夫妻宫) 
 配偶俊美，气质高尚，少说话，行事谨慎，外向，贤惠诚恳，爱家庭，但好打扮，爱听好话。有亲上加亲的说法，配偶是老相识、同事、同学或邻居，或是亲友介绍撮合而成。婚姻多吉利，在失陷时差些。天相与右弼同宫，男命为好格局，配偶环境好，夫荣妻贵；庙旺更佳，若命宫无正星则中途会有变化；女命丈夫虽富贵，不免有偏房的性质，可能为二奶或继室。财荫夹，男得妻家助力，妻子为贤内助。刑忌夹或刑囚夹，配偶事业不利，甚则官司牢灾；又见空劫陀，一生难得婚姻机会，独身主义。火铃同，不投缘，离异。见六吉单星（右弼除外）或见六吉对星又会另一单星者，反主有二婚。男命宜兼看太阴，女命宜兼看太阳，二星吉则和美，凶则有咎。亦宜兼看天府，吉则更吉，凶则更凶。加桃花星，有外遇；加天姚，为色情损名破财。 
 入庙，男女皆有才华，男可配相貌出众而贤惠的妻子，女则配有气质高尚、称心如意的丈夫，配偶宜年长或年龄相差不大。加吉更美。加煞及左辅则不和，甚则有刑克或离异。 
 失陷，夫妻性情不太协调，但能偕老。加吉和谐。加煞及左辅单星则会刑克或离异。 
 丑未宫，配偶能干。男妻子当家作主，有纳妾的可能。女命丈夫事业有成，富贵荣显。加右弼单星吉，加左辅单星反为不好，有第三者插足，易有两度婚姻。煞刑忌重重，终生孤独，有独身倾向。会廉贞化忌，主分离。会贪狼化忌，被人夺爱。加昌曲，生离死别。 
 已亥宫，配偶事业心重，或重朋友情义而冷落娇妻，感情平淡。男命妻子在外多应酬活动。武梁夹或财荫夹，配偶助力大。加煞刑忌主刑克、生离，配偶多灾病。刑忌夹，不利配偶、婚姻。最嫌武曲化忌，主配偶有意外或灾病，不投缘，重则中途生离死别。 
 卯酉宫，感情不和，但可从一而终。加吉男妻为贤妻良母，女命丈夫情深。加煞刑主刑克、生离，配偶多灾病。不喜会廉贪武化忌，最嫌武曲化忌，主配偶贫困或灾病。
----------------------------------------------------------------------------------------------------
Document 2:
☆15、其它星(夫妻宫) 
 文昌一一夫妻恩爱，配偶聪明而有高雅的爱好，动作稍迟钝。本星及主星庙旺，婚姻较理想，加吉更美。紫府相阴阳禄魁钺左右曲同宫，主配偶知识水平高，富贵。左右同宫，妻美，自己有

In [12]:
summary_text = summarize_passages(question, user_chart, retrieved_docs)
summary_text

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'关键信息:\n- 天相星在夫妻宫，配偶性格外向、贤惠，可能是老相识或由亲友介绍，婚姻多吉利。\n- 天相与右弼同宫，男命有良好配偶环境，若命宫无正星则可能会有变化。\n- 夫妻宫中有煞星或刑克，可能导致婚姻不和、分离或多婚。\n- 七杀星在夫妻宫，婚姻不顺，配偶个性强硬，可能导致感情淡薄和争执。\n- 天梁星在夫妻宫，主婚前初恋失败，婚后可能经历波折，但若无煞则可复合。\n- 文曲星在夫妻宫，配偶聪明且有高雅爱好，婚姻较理想，若加煞则可能不和。\n\n命盘关联:\n- 用户命盘中夫妻宫主星为天相，辅星为天钺，符合天相星的描述，可能有良好的配偶环境。\n- 命盘中缺少明显的桃花星，可能影响正缘的出现。\n- 夫妻宫的煞星和刑克情况未详细列出，可能影响婚姻的稳定性。\n\n信息缺口:\n- 夫妻宫的具体煞星和刑克情况缺失，需进一步确认以评估婚姻的潜在风险。\n- 缺少对命宫中桃花星的具体分析，可能影响正缘的出现时间和质量。'

In [13]:
print(summary_text)

关键信息:
- 天相星在夫妻宫，配偶性格外向、贤惠，可能是老相识或由亲友介绍，婚姻多吉利。
- 天相与右弼同宫，男命有良好配偶环境，若命宫无正星则可能会有变化。
- 夫妻宫中有煞星或刑克，可能导致婚姻不和、分离或多婚。
- 七杀星在夫妻宫，婚姻不顺，配偶个性强硬，可能导致感情淡薄和争执。
- 天梁星在夫妻宫，主婚前初恋失败，婚后可能经历波折，但若无煞则可复合。
- 文曲星在夫妻宫，配偶聪明且有高雅爱好，婚姻较理想，若加煞则可能不和。

命盘关联:
- 用户命盘中夫妻宫主星为天相，辅星为天钺，符合天相星的描述，可能有良好的配偶环境。
- 命盘中缺少明显的桃花星，可能影响正缘的出现。
- 夫妻宫的煞星和刑克情况未详细列出，可能影响婚姻的稳定性。

信息缺口:
- 夫妻宫的具体煞星和刑克情况缺失，需进一步确认以评估婚姻的潜在风险。
- 缺少对命宫中桃花星的具体分析，可能影响正缘的出现时间和质量。


In [14]:
final_answer = answer_question(question, user_chart, summary_text)
print('最终回答:', final_answer)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


最终回答: 最终结论: 您在未来的几年内有机会遇到正缘，但需注意婚姻中的潜在波折和不和。

关键理由:
- 您的夫妻宫主星为天相，辅星为天钺，表明您有良好的配偶环境，可能会遇到性格外向、贤惠的伴侣，且婚姻多吉利。
- 然而，命盘中缺少明显的桃花星，这可能影响正缘的出现时间和质量，导致您在感情方面的机会相对较少。
- 夫妻宫中可能存在一些煞星或刑克的情况，这可能会影响婚姻的稳定性，导致感情中的争执和波折。

建议:
- 在未来的1-3年内，积极参与社交活动，扩大人际圈，增加遇到正缘的机会。
- 尽量保持开放的心态，接受朋友或亲友的介绍，可能会有意想不到的收获。
- 在感情中要注意沟通，避免因性格差异导致的争执，尤其是在婚姻初期。
- 如果可能，建议进行命理咨询，进一步分析夫妻宫的煞星和刑克情况，以便更好地应对未来的感情挑战。
